In [8]:
from math import log
import pickle
import operator
#计算基础香农熵
def calcShannonEnt(dateSet):
    numEntries = len(dateSet)
    labelCounts = {}
#判断是正类，还是负类，提取的是yes 和 no
    for featVec in dateSet:
        currentLabel = featVec[-1]
        labelCounts[currentLabel] = labelCounts.get(currentLabel,0)+1
#求香农熵
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntries
        shannonEnt -= prob*log(prob,2)
    return shannonEnt
#划分数据集
def splitDateSet(dateSet,axis,value):
    retDateSet = []
    for featVec in dateSet:
        if featVec[axis] == value:
            #不包含特征axis
            reducedFeatVec=featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDateSet.append(reducedFeatVec)
    return retDateSet
#选择最好的划分数据集的方式，即信息增益最大的特征，作为根特征
def chooseBestFeatureToSplit(dateSet):
    numberFeatures = len(dateSet[0]) - 1#特征个数
    baseEntropy = calcShannonEnt(dateSet)
    bestInfoGain = 0.0
    bestFeature = -1
    #遍历每一个特征,把每个样本中的对应特征挨个提取出来
    for i in range(numberFeatures):
        featList = [example[i] for example in dateSet]
        uniqueVals = set(featList) #某一个特征的取值列表
        newEntropy = 0.0
        #求各个不同x的值分别的熵，再相加
        for value in uniqueVals:
            subDateSet = splitDateSet(dateSet,i,value)
            prob = len(subDateSet)/float(len(dateSet))#每一个xi都要取到
            newEntropy += prob * calcShannonEnt(subDateSet)
        infoGain = baseEntropy - newEntropy
        if infoGain > bestInfoGain:
            bestInfoGain = infoGain
            bestFeature =i
    return bestFeature
#计算正类和负类的出现频率，返回出现频率最高的标签，当遍历完所有属性后，用多数表决来判断该叶节点是正类还是负类
def majorityCnt(classList):
    classCount = {}
    for vote in classList:
        classCount[vote] = classCount.get(vote,0)+1
    sortedClassCount = sorted(classCount.items(),key = lambda x:x[1],reverse=True)
    return sortedClassCount[0][0]
#创建决策树
def creatDateTree(dateSet,labels):
    classList = [example[-1] for example in dateSet]
     #具有相同的分支，不能再通过yes或者no来区别了
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    #属性遍历完了
    if len(dateSet[0]) ==1:
        return majorityCnt[classList]
    bestFeat = chooseBestFeatureToSplit(dateSet)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}#用字典类型储存决策树，字典的一对多的实现，用子字典充当字典的value,不允许值重复
    copyLabels = labels[:]
    del(copyLabels[bestFeat]) #用过的特性就不用了，标签删掉
    featValues = [example[bestFeat] for example in dateSet]
    uniqueValues = set(featValues)
    for value in uniqueValues:
        subLabels = copyLabels
        myTree[bestFeatLabel][value] = creatDateTree(splitDateSet(dateSet,bestFeat,value),subLabels)
    return myTree
#测试决策树
def classify(inputTree,featLabels,testVec):
    firstSide = list(inputTree.keys())
    firstStr = firstSide[0]
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr)
    for key in secondDict.keys():
        if testVec[featIndex] == key:  #标签属性的值如果等于key
            if type(secondDict[key]).__name__=="dict":
                print(secondDict[key])
                classLabel = classify(secondDict[key],featLabels,testVec)
            else:
                classLabel = secondDict[key]
    return classLabel   
#决策树的存储
def storeTree(inputTree,filename):
    fp =open(filename,"wb")
    pickle.dump(inputTree,fp)
    fp.close
def grabTree(filename):
    fp = open(filename,"rb")
    return pickle.load(fp)
if __name__ =="__main__":
    filename ="F:\\python_daima\\KD_Tree\\glasses\\lenses.txt"
    fp = open(filename)
    dateSet = [inst.strip().split("\t") for inst in fp.readlines()]
    labels =["age","prescript","astigmatic","tearRate"]
    myTree = creatDateTree(dateSet,labels)
    print(myTree)
    Label=classify(myTree,labels,["young","myope","no","reduced"])
    print(Label)

    

{'tearRate': {'normal': {'astigmatic': {'yes': {'prescript': {'hyper': {'age': {'presbyopic': 'no lenses', 'pre': 'no lenses', 'young': 'hard'}}, 'myope': 'hard'}}, 'no': {'age': {'presbyopic': {'prescript': {'hyper': 'soft', 'myope': 'no lenses'}}, 'pre': 'soft', 'young': 'soft'}}}}, 'reduced': 'no lenses'}}
no lenses
